## Data

In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 13
sequence_length = 7

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'<unk>': 0,
 'cây': 4,
 'kẻ': 6,
 'thì': 10,
 'chí': 3,
 '<pad>': 1,
 'nhớ': 7,
 '<sos>': 2,
 'có': 5,
 'nên': 8,
 'quả': 9,
 'trồng': 11,
 'ăn': 12}

In [3]:
data_x = []
data_y = []
for vector in corpus:
    vector = vector.split()    

    for i in range(len(vector)):
        data_x.append(vector[:i])
        data_y.append(vector[i])

print(data_x)
print(data_y)

[[], ['ăn'], ['ăn', 'quả'], ['ăn', 'quả', 'nhớ'], ['ăn', 'quả', 'nhớ', 'kẻ'], ['ăn', 'quả', 'nhớ', 'kẻ', 'trồng'], [], ['có'], ['có', 'chí'], ['có', 'chí', 'thì']]
['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', 'có', 'chí', 'thì', 'nên']


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):
    x = ['<sos>'] + x
    print(x)
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))
    
    return x_ids, vocab[y]

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

['<sos>']
['<sos>', 'ăn']
['<sos>', 'ăn', 'quả']
['<sos>', 'ăn', 'quả', 'nhớ']
['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ']
['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng']
['<sos>']
['<sos>', 'có']
['<sos>', 'có', 'chí']
['<sos>', 'có', 'chí', 'thì']


In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 1, 1, 1, 1, 1, 1]
12

[2, 12, 1, 1, 1, 1, 1]
9

[2, 12, 9, 1, 1, 1, 1]
7

[2, 12, 9, 7, 1, 1, 1]
6

[2, 12, 9, 7, 6, 1, 1]
11

[2, 12, 9, 7, 6, 11, 1]
4

[2, 1, 1, 1, 1, 1, 1]
5

[2, 5, 1, 1, 1, 1, 1]
3

[2, 5, 3, 1, 1, 1, 1]
10

[2, 5, 3, 10, 1, 1, 1]
8



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([10, 7])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([10])


## Train with full data

In [8]:
class TransformerDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, 
                                          num_heads=num_heads,
                                          batch_first=True)
        self.ffn = nn.Linear(in_features=embed_dim, 
                             out_features=ff_dim)
        self.layernorm_1 = nn.LayerNorm(normalized_shape=embed_dim)
        self.layernorm_2 = nn.LayerNorm(normalized_shape=embed_dim)

    def forward(self, query, key, value, mask):
        attn_output, _ = self.attn(query, key, value, attn_mask=mask)
        out_1 = self.layernorm_1(query + attn_output)
        ffn_output = nn.GELU()(self.ffn(out_1))
        out_2 = self.layernorm_2(out_1 + ffn_output)
        return out_2

# test
transformer = TransformerDecoderBlock(6, 2, 6)
t = torch.randn(1, 10, 6)
mask = torch.triu(torch.ones(10, 10), diagonal=1).bool()  
output = transformer(t, t, t, mask)
print(output.shape)

torch.Size([1, 10, 6])


In [9]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.mask = torch.triu(torch.ones(sequence_length, sequence_length), 
                               diagonal=1).bool()
        self.transformer = TransformerDecoderBlock(embed_dim, 
                                                   num_heads, 
                                                   embed_dim)
        self.linear = nn.Linear(sequence_length*embed_dim, 
                                vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x, x, self.mask)
        x = nn.Flatten()(x)
        x = self.linear(x)
        return x

model = TG_Model(vocab_size, 8, 2, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(13, 8)
  (transformer): TransformerDecoderBlock(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
    )
    (ffn): Linear(in_features=8, out_features=8, bias=True)
    (layernorm_1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (layernorm_2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
  )
  (linear): Linear(in_features=56, out_features=13, bias=True)
)


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [11]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.583068609237671
1.827111840248108
1.3027745485305786
0.7586214542388916
0.5297244787216187
0.35368064045906067
0.2713151276111603
0.19679014384746552
0.17545171082019806
0.15838469564914703
0.14841589331626892
0.14442755281925201
0.14147578179836273
0.14039680361747742
0.13998113572597504
0.141950324177742
0.16292119026184082
0.20254306495189667
0.14872241020202637
0.16309711337089539
0.14708758890628815
0.15652121603488922
0.14150211215019226
0.1535278558731079
0.1387092024087906
0.15102869272232056
0.14065925776958466
0.1448163092136383
0.14456884562969208
0.1395975649356842
0.1458607017993927
0.13902555406093597
0.14310532808303833
0.14144526422023773
0.139687180519104
0.14284059405326843
0.13868984580039978
0.14173725247383118
0.13983085751533508
0.13972818851470947


In [12]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=-1))

tensor([12,  9,  7,  6, 11,  4, 12,  3, 10,  8])


In [13]:
data_y_ids

tensor([12,  9,  7,  6, 11,  4,  5,  3, 10,  8])

## Inference

In [14]:
promt = '<sos>'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[2, 1, 1, 1, 1, 1, 1]


In [15]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    next_id = torch.argmax(outputs, axis=-1)

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[2, 12, 1, 1, 1, 1, 1]
[2, 12, 9, 1, 1, 1, 1]
[2, 12, 9, 7, 1, 1, 1]
[2, 12, 9, 7, 6, 1, 1]
[2, 12, 9, 7, 6, 11, 1]
[2, 12, 9, 7, 6, 11, 4]
